In [ ]:
# https://developers.google.com/drive/api/v3/ref-search-terms
# https://developers.google.com/drive/api/v3/search-files


In [ ]:
from __future__ import print_function
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from googleapiclient import discovery
from googleapiclient.errors import HttpError
import json

import pandas as pd


# NLP libraries for keywords and summaries
import re
import nltk
from nltk.probability import FreqDist
import matplotlib.pyplot as plt
#import heapq
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS


In [ ]:
SCOPES = ['https://www.googleapis.com/auth/documents.readonly', 
          'https://www.googleapis.com/auth/documents',
          'https://www.googleapis.com/auth/drive.metadata.readonly',
          'https://www.googleapis.com/auth/drive',
          'https://www.googleapis.com/auth/drive.file',
          'https://www.googleapis.com/auth/drive.readonly'
         ]


In [ ]:
def getCreds(scopes):
    creds = None
    # The file token.json stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', scopes)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', scopes)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.json', 'w') as token:
            token.write(creds.to_json())
    return(creds)

def searchGDrive(query, creds, debug=False):
    DRIVE = discovery.build('drive', 'v3', credentials=creds)
    retError = ""
    errorState = False
    try:
        fileList = DRIVE.files().list(q=query).execute()
    except HttpError as ex:
        errorState = True
        template = "An exception of type {0} occurred. Arguments:\n{1!r}"
        message = template.format(type(ex).__name__, ex.args)
        if (debug):
            print (message)
        retError = str(ex.resp.status)
        if ex.resp.get('content-type', '').startswith('application/json'):
            reason = json.loads(ex.content).get('error').get('errors')[0].get('reason')
            saveString = json.loads(ex.content)
            if (debug):
                print (ex.resp.status, reason)
            retError = str(ex.resp.status) + ' - ' + reason
    
    if (errorState or (not(fileList and 'files' in fileList))):
        if (debug):
            print ('Error')
        return(False, pd.DataFrame(), retError)

    files_df = pd.DataFrame(pd.json_normalize(fileList))
    fileCount = len(files_df.index)
    if (len(files_df.index) == 0):
        return (False, files_df, 'Nothing Found')
    return (True, files_df, '')

def getDocumentFromID(docID, creds, debug=False):
    service = build('docs', 'v1', credentials=creds)

    retError = ""
    errorState = False
    try:
        document = service.documents().get(documentId=docID).execute()
    except HttpError as ex:
        errorState = True
        template = "An exception of type {0} occurred. Arguments:\n{1!r}"
        message = template.format(type(ex).__name__, ex.args)
        if (debug):
            print (message)
        retError = str(ex.resp.status)
        if ex.resp.get('content-type', '').startswith('application/json'):
            reason = json.loads(ex.content).get('error').get('errors')[0].get('reason')
            saveString = json.loads(ex.content)
            if (debug):
                print (ex.resp.status, reason)
            retError = str(ex.resp.status) + ' - ' + reason
    
    if (errorState or (not(document))):
        if (debug):
            print ('Error')
        return(False, {}, retError)
    
    return(True, document, retError)


def googleDriveInfo(fileID, creds, debug=False):
    DRIVE = discovery.build('drive', 'v3', credentials=creds)

    # Drive V3 API:
    #
    #try:
    saveString = ""
    retError = ""
    errorState = False
    try:
        cols = 'name,mimeType,owners,createdTime,modifiedTime,shared,permissions'
        fileInfo = DRIVE.files().get(fileId=fileID,fields=cols).execute()
    except HttpError as ex:
        errorState = True
        template = "An exception of type {0} occurred. Arguments:\n{1!r}"
        message = template.format(type(ex).__name__, ex.args)
        if (debug):
            print (message)
        retError = str(ex.resp.status)
        if ex.resp.get('content-type', '').startswith('application/json'):
            reason = json.loads(ex.content).get('error').get('errors')[0].get('reason')
            saveString = json.loads(ex.content)
            if (debug):
                print (ex.resp.status, reason)
            retError = str(ex.resp.status) + ' - ' + reason
        
    if (errorState or not(fileInfo)):
        return(False, retError)
    
    return (True, fileInfo)

def googleDriveText(fileID, creds, debug=False):
    DRIVE = discovery.build('drive', 'v3', credentials=creds)
    retError = ""
    errorState = False
    try:
        fileText = DRIVE.files().export(fileId=fileID, mimeType='text/plain').execute()
    except HttpError as ex:
        errorState = True
        template = "An exception of type {0} occurred. Arguments:\n{1!r}"
        message = template.format(type(ex).__name__, ex.args)
        if (debug):
            print (message)
        retError = str(ex.resp.status)
        if ex.resp.get('content-type', '').startswith('application/json'):
            reason = json.loads(ex.content).get('error').get('errors')[0].get('reason')
            saveString = json.loads(ex.content)
            if (debug):
                print (ex.resp.status, reason)
            retError = str(ex.resp.status) + ' - ' + reason
    
    if errorState:
        if (debug):
            print ('Error')
        return(False, retError)
    else:
        if (debug):
            print (len(fileText.decode('utf-8')))
        return(True, fileText.decode('utf-8'))

def getSummary(document, keywordCount = 5, debug=False):
    # Removing Square Brackets and Extra Spaces
    article_text = re.sub(r'\[[0-9]*\]', ' ', document)
    article_text = re.sub(r'\s+', ' ', article_text)
    # Removing special characters and digits
    formatted_article_text = re.sub('[^a-zA-Z]', ' ', article_text )
    formatted_article_text = re.sub(r'\s+', ' ', formatted_article_text)
    sentence_list = nltk.sent_tokenize(article_text)
    stopwords = list(ENGLISH_STOP_WORDS)
    
    word_frequencies = {}
    for word in nltk.word_tokenize(formatted_article_text):
        if word not in stopwords:
            if word not in word_frequencies.keys():
                word_frequencies[word] = 1
            else:
                word_frequencies[word] += 1
        maximum_frequncy = 0
        if len(word_frequencies) > 0:
            maximum_frequncy = max(word_frequencies.values())
        else:
            return('', '')
    for word in word_frequencies.keys():
        word_frequencies[word] = (word_frequencies[word]/maximum_frequncy)
        sentence_scores = {}
    for sent in sentence_list:
        for word in nltk.word_tokenize(sent.lower()):
            if word in word_frequencies.keys():
                if len(sent.split(' ')) < 30:
                    if sent not in sentence_scores.keys():
                        sentence_scores[sent] = word_frequencies[word]
                    else:
                        sentence_scores[sent] += word_frequencies[word]
                        
    wordslist = nltk.word_tokenize(formatted_article_text)
    filtered_words = [words for words in wordslist if words.lower() not in stopwords]

    fdist = FreqDist(filtered_words)
    frequent_words = [[fdist[word], word] for word in set(filtered_words) if len(word) > 2 and fdist[word] >= 2]

    #
    # Record the frequency count of
    #
    sorted_word_frequencies = {}
    for item in sorted(frequent_words):
        sorted_word_frequencies[item[1]] = item[0]
    
    interestingKeywords = list(sorted_word_frequencies.keys())[-keywordCount:]
        
    #summary_sentences = heapq.nlargest(3, sentence_scores, key=sentence_scores.get)
    #summary = ' '.join(summary_sentences)
    return (interestingKeywords, '' )


def getCommentInfo(fileID, creds, debug=False):
    DRIVE = discovery.build('drive', 'v3', credentials=creds)
    retError = ""
    errorState = False
    try:
        commentList = DRIVE.comments().list(fileId=fileID, fields='comments').execute()
    except HttpError as ex:
        errorState = True
        template = "An exception of type {0} occurred. Arguments:\n{1!r}"
        message = template.format(type(ex).__name__, ex.args)
        if (debug):
            print (message)
        retError = str(ex.resp.status)
        if ex.resp.get('content-type', '').startswith('application/json'):
            reason = json.loads(ex.content).get('error').get('errors')[0].get('reason')
            saveString = json.loads(ex.content)
            if (debug):
                print (ex.resp.status, reason)
            retError = str(ex.resp.status) + ' - ' + reason
    
    if (errorState or (not(commentList and 'comments' in commentList))):
        if (debug):
            print ('Error')
        return(False, 0, retError)

    comment_df = pd.DataFrame(pd.json_normalize(commentList['comments']))
    if (len(comment_df.index) == 0) or ('id' not in comment_df) :
        return (False, 0, None)
    
    commentCount = 0
    if 'id' in comment_df:
        commentCount = comment_df['id'].nunique()
        
    resolvedCount = 0
    if 'resolved' in comment_df:
        resolvedCount = sum(comment_df['resolved'].notna())
    
    reply_df = pd.DataFrame(pd.json_normalize(commentList['comments'], record_path=['replies']))
    resolvedAction = 0
    if 'action' in reply_df:
        resolvedAction = sum(reply_df['action'] == 'resolve')

    totalReplies = 0
    if ('id' in reply_df):
        totalReplies = reply_df['id'].nunique()
    
    commenters = []
    if 'author.displayName' in reply_df:
        commenters = list(reply_df['author.displayName'].unique())
    
    resultDic = {"commentCount": commentCount, 
                "resolvedCount": resolvedCount, # comments that were resolved
                "resolvedActions" : resolvedAction, #hit resolve on a comment (w/o text/response)
                "totalReplies" : totalReplies,
                "commenters" : commenters }
    if (debug):
        print (resultDic)
    return(True, commentCount, resultDic)


def getDocumentFromID(docID, creds, debug=False):
    service = build('docs', 'v1', credentials=creds)

    retError = ""
    errorState = False
    try:
        document = service.documents().get(documentId=docID).execute()
    except HttpError as ex:
        errorState = True
        template = "An exception of type {0} occurred. Arguments:\n{1!r}"
        message = template.format(type(ex).__name__, ex.args)
        if (debug):
            print (message)
        retError = str(ex.resp.status)
        if ex.resp.get('content-type', '').startswith('application/json'):
            reason = json.loads(ex.content).get('error').get('errors')[0].get('reason')
            saveString = json.loads(ex.content)
            if (debug):
                print (ex.resp.status, reason)
            retError = str(ex.resp.status) + ' - ' + reason
    
    if (errorState or (not(document))):
        if (debug):
            print ('Error')
        return(False, {}, retError)
    
    return(True, document, retError)

def getDocTitleFromID(docID, creds, debug=False):
    (res, jsonDoc, errText) = getDocumentFromID(docID, creds, debug)
    if (not res):
        return (res, jsonDoc, errText)
    elif ('title' in jsonDoc) and ('documentId' in jsonDoc):
        if (debug):
            print ('found title and documentId..')
        return(res, jsonDoc['title'], jsonDoc['documentId'])
    else:
        if (debug):
            print ('fail not found')
        return(res, '','')

def getFileListing(in_df,  searchString, searchTopic, searchTag='ZZZZZZ', debug=False):
    if (debug and 'id' in in_df):
        print (len(in_df.index), in_df['id'].nunique())

    for t in searchTopic:
        qString = searchString.replace(searchTag, t)
        (res, result_df, errTxt) = searchGDrive(qString, creds, debug=False)
        n = 0
        if (res) and (len(result_df.index) > 0) :
            if 'files' in result_df:
                temp_df = pd.DataFrame(pd.json_normalize(result_df['files']).T)
                new_df = pd.DataFrame(list(temp_df[0]))
                n = len(new_df.index)
                if (debug):
                    print ('found: '+t+', results = '+str(n))
                new_df['searchTerm'] = t
                append_df = new_df.copy()
                # only put in unique file entries (e.g. a search may find a file multiple times)
                #
                if ('id' in in_df and 'id' in new_df):
                    
                    l = ~new_df.id.isin(list(in_df['id'].unique()))
                    if (debug) and (sum(l) != len(new_df.index)):
                        print('Found Duplicates', sum(l), len(new_df.index))

                    append_df = new_df.loc[l,].copy()
                    
                if (debug):
                    print ('appending:', len(append_df.index))
                in_df = pd.concat([in_df, append_df], ignore_index=True)
    if (debug):
        print (len(in_df.index), in_df['id'].nunique())
    return(in_df)

## Query Examples:
  * query = "mimeType='application/vnd.google-apps.document' and trashed=false and name contains 'Kafka'"
  * query = "modifiedDate > '2021-06-04T12:00:00'"
  * query = "mimeType='application/vnd.google-apps.document' and trashed=false and name contains 'Kafka' and fullText contains 'scale' and 'user@host.com' in writers"


In [ ]:
creds = getCreds(SCOPES)

## Search by Topic String in File Names

In [ ]:
titleStrings = ['design', 'requirements', 'prd', 'spec', 'proposal', 'decision']
searchTag = 'ZZZZZZ'
query = "mimeType='application/vnd.google-apps.document' and trashed=false and name contains '"+searchTag+"' and fullText contains 'Kafka' and createdTime > '2021-01-01T03:09:26.605Z'"

files_df = pd.DataFrame()
files_df = getFileListing(files_df,  query, titleStrings, searchTag, False)

## Search by PM writer

In [ ]:
pmStrings = ['first.last', 'another_user']
searchTag = 'ZZZZZZ'
query = "mimeType='application/vnd.google-apps.document' and trashed=false and name contains 'Kafka' and fullText contains 'Interesting' and 'ZZZZZZ@user.com' in writers"

files_df = getFileListing(files_df,  query, pmStrings, searchTag, False)
save_files_df = files_df.copy()
print (len(save_files_df.index), len(files_df.index), files_df['id'].nunique())

In [ ]:
#save_files_df['id'].iloc[35:40]

## Get title, comments and other metadata for the GoogleDoc

In [ ]:
files_df['commentCount'] = 0
files_df['totalReplies'] = 0
files_df['commenters'] = ''
files_df['keywords'] = ''
files_df['size'] = 0
files_df['wordcount'] = 0
baseURL = 'https://docs.google.com/document/d/'
files_df['url']  = baseURL + files_df['id'] + '/preview'
keywordCount = 7
debugString = True

fileProcessed = 0
#for index, row in files_df.iloc[:12].iterrows():
for index, row in files_df.iterrows():
    (res, commentCount, commentDic) = getCommentInfo(row['id'], creds, debug=False)
    if (debugString):
        print (fileProcessed, index, 'DocID:', row['id'])
    fileProcessed = fileProcessed + 1
    if (res and commentCount > 0):
        # print (commentDic['commentCount'], commentDic['totalReplies'], commentDic['commenters'])
        files_df.at[index,'commentCount'] = commentDic['commentCount']
        files_df.at[index,'totalReplies'] = commentDic['totalReplies']
        if 'commenters' in commentDic:
            #files_df.at[index,'commenters'] = [','.join(map(str, l)) for l in commentDic['commenters']]
            # print (', '.join(commentDic['commenters']))
            files_df.at[index,'commenters']  = ', '.join(commentDic['commenters'])
            
    (res, titleStr, idStr) = getDocTitleFromID(row['id'], creds, debug=False)
    if (res and len(titleStr) > 0):
        files_df.at[index,'title'] = titleStr
        
    (res, fileText) = googleDriveText(row['id'], creds)

    
    if (res and len(fileText) > 0):
        (kws, summaryText) = getSummary(fileText, keywordCount)
        if (len(kws) > 0):
            files_df.at[index,'keywords'] = kws
        files_df.at[index,'wordcount'] = len(fileText.split())
        files_df.at[index,'size'] = len(fileText)


In [ ]:
#len(fileText.split())

In [ ]:
print (list(files_df.columns))
files_df.head()

In [ ]:
cols = list(files_df.columns)
if 'mimeType' in cols:
    cols.remove('mimeType')
if 'kind' in cols:
    cols.remove('kind')
files_df[cols].head()

In [ ]:
csv_outfile = 'search_index.csv'
files_df[cols].to_csv(csv_outfile, index=False)
n=len(files_df.index)
print ('wrote: '+ str(n) +  ' lines to ' + csv_outfile)

# Done - ZZZZ